In [1]:
!pip install -e .

Obtaining file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md
ERROR: file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# py-tidymodels imports
from py_workflows import workflow
from py_parsnip import linear_reg
from py_rsample import vfold_cv, initial_split, training, testing
from py_yardstick import metric_set, rmse, mae, r_squared, smape
from py_workflowsets import WorkflowSet

from py_recipes import recipe
from py_recipes.selectors import all_numeric, all_nominal, starts_with, ends_with

# Set random seed for reproducibility
np.random.seed(42)

# Import Data

In [3]:
# import raw data
raw_data = pd.read_csv('__data/preem.csv')

In [4]:
df = raw_data

# set date as index
#df = raw_data.set_index('date')

# convert date to datetime
df['date'] = pd.to_datetime(df['date'])

In [5]:
df

,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2020-04-01,-56.18,-11.43,-3.07,28.58,-16.09,26.91,0.47,137.65,0.00
1,2020-05-01,-42.36,-9.17,3.89,22.64,-8.75,20.36,0.57,113.53,0.00
2,2020-06-01,-30.21,-8.58,-3.44,13.43,-4.28,11.32,6.61,43.31,0.00
3,2020-07-01,-28.86,-6.86,-2.71,10.74,-3.20,8.53,-1.55,79.77,0.00
4,2020-08-01,-38.80,-5.86,-2.63,4.58,-5.35,6.26,-2.70,48.27,0.00
5,2020-09-01,-41.76,-5.62,-2.08,7.69,-3.26,6.15,-2.10,98.01,0.00
6,2020-10-01,-43.99,-5.10,0.16,8.09,-4.31,5.80,-3.34,49.77,0.00
7,2020-11-01,-42.94,-5.43,1.80,7.86,-7.40,6.99,-2.53,35.48,0.00
8,2020-12-01,-41.55,-3.39,0.94,6.68,-7.65,6.22,0.00,63.98,0.00
9,2021-01-01,-41.18,-6.03,0.63,8.67,-7.67,8.22,-0.05,109.76,0.00


# py-rsample

In [6]:
from py_rsample import initial_time_split, time_series_cv

In [7]:
# Create train/test split
split = initial_split(df, prop=0.75, seed=123)
train_data = training(split)
test_data = testing(split)

print(f"Training set: {train_data.shape[0]} rows")
print(f"Test set: {test_data.shape[0]} rows")

Training set: 42 rows
Test set: 15 rows


In [8]:
# Example: Train on 2020, test on first half of 2021
split_absolute = initial_time_split(
    df,
    date_column="date",
    train_start="2022-01-01",
    train_end="2023-12-01",
    test_start="2024-01-01",
    test_end="2024-06-01"
)

train_abs = split_absolute.training()
test_abs = split_absolute.testing()

print(f"Training: {len(train_abs)} days from {train_abs['date'].min()} to {train_abs['date'].max()}")
print(f"Testing:  {len(test_abs)} days from {test_abs['date'].min()} to {test_abs['date'].max()}")

# bug - thinks its days not months (monthly data)

Training: 24 days from 2022-01-01 00:00:00 to 2023-12-01 00:00:00
Testing:  6 days from 2024-01-01 00:00:00 to 2024-06-01 00:00:00


In [9]:
# Example 1: Train on first 18 months, test on last 6 months
split_relative1 = initial_time_split(
    df,
    date_column="date",
    train_start="start",
    train_end="start + 18 months",
    test_start="end - 6 months",
    test_end="end"
)

train_rel1 = split_relative1.training()
test_rel1 = split_relative1.testing()

print("Example 1: First 18 months vs last 6 months")
print(f"Training: {len(train_rel1)} days from {train_rel1['date'].min()} to {train_rel1['date'].max()}")
print(f"Testing:  {len(test_rel1)} days from {test_rel1['date'].min()} to {test_rel1['date'].max()}")
print()

# bug - thinks its days not months (monthly data)

Example 1: First 18 months vs last 6 months
Training: 18 days from 2020-04-01 00:00:00 to 2021-09-01 00:00:00
Testing:  6 days from 2024-07-01 00:00:00 to 2024-12-01 00:00:00



In [10]:
# Train on specific year (2020), test on last 3 months of available data
split_mixed = initial_time_split(
    df,
    date_column="date",
    train_start="start",
    train_end="2023-12-01",
    test_start="end - 12 months",
    test_end="end"
)

train_mix = split_mixed.training()
test_mix = split_mixed.testing()

print("Mixed: Train on 2020, test on last 90 days of data")
print(f"Training: {len(train_mix)} days from {train_mix['date'].min()} to {train_mix['date'].max()}")
print(f"Testing:  {len(test_mix)} days from {test_mix['date'].min()} to {test_mix['date'].max()}")

# bug - thinks its days not months (monthly data)

Mixed: Train on 2020, test on last 90 days of data
Training: 45 days from 2020-04-01 00:00:00 to 2023-12-01 00:00:00
Testing:  12 days from 2024-01-01 00:00:00 to 2024-12-01 00:00:00


# 1. Single Model Fitting

In [11]:
FORMULA_STR = "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1"

## 1.1 - Prophet

In [12]:
# SINGLE MODEL fitting

from py_parsnip import prophet_reg, arima_reg, rand_forest, linear_reg

# Create Prophet specification
# Prophet will automatically detect weekly seasonality in our data
spec_prophet = prophet_reg(
    n_changepoints=25,  # Number of potential trend changes
    changepoint_prior_scale=0.05,  # Flexibility of trend
    seasonality_prior_scale=10.0   # Flexibility of seasonality
)

print(spec_prophet)

# # Fit Prophet - FAILING
fit_prophet = spec_prophet.fit(train_data, FORMULA_STR)
print("Prophet model fitted!")

# Evaluate and extract outputs
fit_prophet = fit_prophet.evaluate(test_data)
outputs_prophet, coefs_prophet, stats_prophet = fit_prophet.extract_outputs()

print("Prophet OUTPUTS:")
print(outputs_prophet[outputs_prophet['split'] == 'test'].head(10))

ModelSpec(model_type='prophet_reg', engine='prophet', mode='regression', args={'growth': 'linear', 'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10.0, 'seasonality_mode': 'additive', 'n_changepoints': 25, 'changepoint_range': 0.8}, date_col=None)


07:59:03 - cmdstanpy - INFO - Chain [1] start processing
07:59:03 - cmdstanpy - INFO - Chain [1] done processing


Prophet model fitted!
Prophet OUTPUTS:
         date  actuals      fitted  forecast  residuals split        model  \
42 2023-10-01   107.65   63.911173    107.65  43.738827  test  prophet_reg   
43 2023-11-01   118.37  205.128723    118.37 -86.758723  test  prophet_reg   
44 2023-12-01   113.03  153.708332    113.03 -40.678332  test  prophet_reg   
45 2024-01-01   122.65   61.631319    122.65  61.018681  test  prophet_reg   
46 2024-02-01   129.09   72.224310    129.09  56.865690  test  prophet_reg   
47 2024-03-01   145.57   88.385033    145.57  57.184967  test  prophet_reg   
48 2024-04-01   158.23  151.980701    158.23   6.249299  test  prophet_reg   
49 2024-05-01   104.04  147.915728    104.04 -43.875728  test  prophet_reg   
50 2024-06-01   173.23  101.414406    173.23  71.815594  test  prophet_reg   
51 2024-07-01   126.20  104.170767    126.20  22.029233  test  prophet_reg   

   model_group_name   group  
42                   global  
43                   global  
44          

In [13]:
coefs_prophet

,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


In [14]:
outputs_prophet

,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,115.345206,137.65,22.304794,train,prophet_reg,,global
1,2020-05-01,113.53,116.260686,113.53,-2.730686,train,prophet_reg,,global
2,2020-06-01,43.31,66.456243,43.31,-23.146243,train,prophet_reg,,global
3,2020-07-01,79.77,69.442704,79.77,10.327296,train,prophet_reg,,global
4,2020-08-01,48.27,63.233826,48.27,-14.963826,train,prophet_reg,,global
5,2020-09-01,98.01,111.587870,98.01,-13.577870,train,prophet_reg,,global
6,2020-10-01,49.77,78.158785,49.77,-28.388785,train,prophet_reg,,global
7,2020-11-01,35.48,56.260693,35.48,-20.780693,train,prophet_reg,,global
8,2020-12-01,63.98,97.418839,63.98,-33.438839,train,prophet_reg,,global
9,2021-01-01,109.76,115.062935,109.76,-5.302935,train,prophet_reg,,global


## 1.2 - ARIMA

In [15]:
# Create ARIMA specification
spec_arima = arima_reg(
    seasonal_period=7,  # Weekly seasonality
    non_seasonal_ar=1,
    non_seasonal_differences=1,
    non_seasonal_ma=1,
    seasonal_ar=1,
    seasonal_differences=0,
    seasonal_ma=1
)

print(spec_arima)

# Fit ARIMA
fit_arima = spec_arima.fit(train_data, FORMULA_STR)
print("ARIMA model fitted!")

# Predict on test data
pred_arima = fit_arima.predict(test_data)
print(pred_arima.head(10))

# Evaluate and extract outputs
fit_arima = fit_arima.evaluate(test_data)
outputs_arima, coefs_arima, stats_arima = fit_arima.extract_outputs()

# Get test metrics
arima_test_metrics = stats_arima[
    (stats_arima['split'] == 'test') & 
    (stats_arima['metric'].isin(['rmse', 'mae', 'mape', 'r_squared']))
][['metric', 'value']]

print("ARIMA Test Metrics:")
print(arima_test_metrics)

ModelSpec(model_type='arima_reg', engine='statsmodels', mode='regression', args={'seasonal_period': 7, 'non_seasonal_ar': 1, 'non_seasonal_differences': 1, 'non_seasonal_ma': 1, 'seasonal_ar': 1, 'seasonal_differences': 0, 'seasonal_ma': 1}, date_col=None)
ARIMA model fitted!
                 .pred
date                  
2023-10-01  104.714398
2023-11-01  113.211972
2023-12-01  103.169142
2024-01-01   97.988539
2024-02-01  102.481876
2024-03-01  105.810258
2024-04-01   91.631446
2024-05-01  102.281171
2024-06-01  103.095116
2024-07-01  104.063084
ARIMA Test Metrics:
       metric      value
6        rmse  30.682114
7         mae  22.253146
8        mape  15.876808
10  r_squared  -1.434941


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [16]:
outputs_arima

,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,3.855227,137.65,133.794773,train,arima_reg,,global
1,2020-05-01,113.53,157.164380,113.53,-43.634380,train,arima_reg,,global
2,2020-06-01,43.31,106.187323,43.31,-62.877323,train,arima_reg,,global
3,2020-07-01,79.77,21.646072,79.77,58.123928,train,arima_reg,,global
4,2020-08-01,48.27,90.761422,48.27,-42.491422,train,arima_reg,,global
5,2020-09-01,98.01,38.766571,98.01,59.243429,train,arima_reg,,global
6,2020-10-01,49.77,112.939793,49.77,-63.169793,train,arima_reg,,global
7,2020-11-01,35.48,29.950048,35.48,5.529952,train,arima_reg,,global
8,2020-12-01,63.98,34.208322,63.98,29.771678,train,arima_reg,,global
9,2021-01-01,109.76,79.044244,109.76,30.715756,train,arima_reg,,global


In [17]:
coefs_arima

,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,mean_med_diesel_crack_input1_trade_month_lag2,-0.011133,1.125936e-01,-0.098881,0.921233,-2.318127e-01,2.095460e-01,NaN,arima_reg,,global
1,mean_nwe_hsfo_crack_trade_month_lag1,-0.282568,5.080476e-01,-0.556184,0.578085,-1.278323e+00,7.131868e-01,NaN,arima_reg,,global
2,ar.L1,0.303552,3.657721e-01,0.829894,0.406599,-4.133479e-01,1.020452e+00,NaN,arima_reg,,global
3,ma.L1,-1.000005,2.857301e+03,-0.000350,0.999721,-5.601206e+03,5.599206e+03,NaN,arima_reg,,global
4,ar.S.L7,-0.088443,4.785682e-01,-0.184807,0.853381,-1.026419e+00,8.495340e-01,NaN,arima_reg,,global
5,ma.S.L7,0.256774,6.921468e-01,0.370982,0.710651,-1.099809e+00,1.613357e+00,NaN,arima_reg,,global
6,sigma2,970.189205,2.772032e+06,0.000350,0.999721,-5.432112e+06,5.434053e+06,NaN,arima_reg,,global


## 1.3 - OLS - Statsmodels

In [18]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_sm = linear_reg().set_engine("statsmodels")

# Fit model
fit_sm = spec_sm.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_sm = fit_sm.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_sm, coefs_sm, stats_sm = fit_sm.extract_outputs()

Model fitted successfully!


In [19]:
outputs_sm

,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,100.051748,137.65,37.598252,train,linear_reg,,global
1,2020-05-01,113.53,97.500057,113.53,16.029943,train,linear_reg,,global
2,2020-06-01,43.31,95.894719,43.31,-52.584719,train,linear_reg,,global
3,2020-07-01,79.77,94.960713,79.77,-15.190713,train,linear_reg,,global
4,2020-08-01,48.27,95.596865,48.27,-47.326865,train,linear_reg,,global
5,2020-09-01,98.01,95.812695,98.01,2.197305,train,linear_reg,,global
6,2020-10-01,49.77,95.820379,49.77,-46.050379,train,linear_reg,,global
7,2020-11-01,35.48,95.855655,35.48,-60.375655,train,linear_reg,,global
8,2020-12-01,63.98,94.771097,63.98,-30.791097,train,linear_reg,,global
9,2021-01-01,109.76,95.936185,109.76,13.823815,train,linear_reg,,global


In [20]:
coefs_sm

,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,88.654446,8.739856,10.143696,1.705717e-12,70.976418,106.332473,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.109948,0.103966,-1.057537,2.967732e-01,-0.320239,0.100343,1.005038,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.456730,0.447737,-1.020085,3.139744e-01,-1.362365,0.448904,1.005038,linear_reg,,global


## 1.4 - OLS - sklearn

In [21]:
# sklearn engine (default)
spec_sk = linear_reg()
fit_sk = spec_sk.fit(train_data, FORMULA_STR)
fit_sk = fit_sk.evaluate(test_data)

print("sklearn model fitted successfully!")

outputs_sk, coefs_sk, stats_sk = fit_sk.extract_outputs()

sklearn model fitted successfully!


In [22]:
outputs_sk

,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,100.051748,137.65,137.650000,train,linear_reg,,global
1,2020-05-01,113.53,97.500057,113.53,113.530000,train,linear_reg,,global
2,2020-06-01,43.31,95.894719,43.31,43.310000,train,linear_reg,,global
3,2020-07-01,79.77,94.960713,79.77,79.770000,train,linear_reg,,global
4,2020-08-01,48.27,95.596865,48.27,48.270000,train,linear_reg,,global
5,2020-09-01,98.01,95.812695,98.01,98.010000,train,linear_reg,,global
6,2020-10-01,49.77,95.820379,49.77,49.770000,train,linear_reg,,global
7,2020-11-01,35.48,95.855655,35.48,35.480000,train,linear_reg,,global
8,2020-12-01,63.98,94.771097,63.98,63.980000,train,linear_reg,,global
9,2021-01-01,109.76,95.936185,109.76,109.760000,train,linear_reg,,global


In [23]:
# Create a simple recipe
rec = (
    recipe()
    .step_normalize()  # Normalize numeric columns
)

# Fit on training data
rec_fit = rec.prep(train_data)

# Apply to training data
train_processed = rec_fit.bake(train_data)
print("\nProcessed training data:")
print(train_processed)

# Apply to test data (using training parameters!)
test_processed = rec_fit.bake(test_data)
print("\nProcessed test data:")
print(test_processed)


Processed training data:
         date  mean_med_diesel_crack_input1_trade_month_lag2  \
0  2020-04-01                                      -0.088770   
1  2020-05-01                                       0.172544   
2  2020-06-01                                       0.402280   
3  2020-07-01                                       0.427807   
4  2020-08-01                                       0.239857   
5  2020-09-01                                       0.183889   
6  2020-10-01                                       0.141723   
7  2020-11-01                                       0.161577   
8  2020-12-01                                       0.187859   
9  2021-01-01                                       0.194855   
10 2021-02-01                                       1.891694   
11 2021-03-01                                      -0.030154   
12 2021-04-01                                       2.080778   
13 2021-05-01                                      -0.271425   
14 2021-06-01 

## 2.1 - Workflows

In [24]:
# Create a workflow with formula and linear regression
wf_ols_sk = (
    workflow()
    .add_formula(FORMULA_STR)
    .add_model(linear_reg().set_engine("sklearn"))
)

print("Workflow components:")
print(f"  Preprocessor: {wf_ols_sk.preprocessor}")
print(f"  Model: {wf_ols_sk.spec.mode} with {wf_ols_sk.spec.engine} engine")

# Fit the workflow
wf_ols_sk_fit = wf_ols_sk.fit(train_data)

print("Workflow fitted successfully!")
print(f"Type: {type(wf_ols_sk_fit)}")

# Predict on test data
predictions =  wf_ols_sk_fit.predict(test_data)

print(f"Predictions shape: {predictions.shape}")
predictions.head()

# Evaluate on test data
wf_ols_sk_fit = wf_ols_sk_fit.evaluate(test_data)

print("Evaluation complete!")
print(f"Evaluation data keys: {list(wf_ols_sk_fit.fit.evaluation_data.keys())}")

# Extract all outputs
outputs, coefficients, stats = wf_ols_sk_fit.extract_outputs()

print("\n=== OUTPUTS (first 10 rows) ===")
print(outputs.head(10))

print("\n=== COEFFICIENTS ===")
print(coefficients)

print("\n=== STATS ===")
print(stats)

Workflow components:
  Preprocessor: target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1
  Model: regression with sklearn engine
Workflow fitted successfully!
Type: <class 'py_workflows.workflow.WorkflowFit'>
Predictions shape: (15, 1)
Evaluation complete!
Evaluation data keys: ['test_data', 'test_predictions', 'outcome_col', 'original_test_data']

=== OUTPUTS (first 10 rows) ===
        date  actuals      fitted  forecast  residuals  split       model  \
0 2020-04-01   137.65  100.051748    137.65     137.65  train  linear_reg   
1 2020-05-01   113.53   97.500057    113.53     113.53  train  linear_reg   
2 2020-06-01    43.31   95.894719     43.31      43.31  train  linear_reg   
3 2020-07-01    79.77   94.960713     79.77      79.77  train  linear_reg   
4 2020-08-01    48.27   95.596865     48.27      48.27  train  linear_reg   
5 2020-09-01    98.01   95.812695     98.01      98.01  train  linear_reg   
6 2020-10-01    49.77   95.820379   

In [25]:
# method chaining
# Full pipeline in one chain
outputs2, coefficients2, stats2 = (
    workflow()
    .add_formula(FORMULA_STR)
    .add_model(linear_reg().set_engine("sklearn"))
    .fit(train_data)
    .evaluate(test_data)
    .extract_outputs()
)

print("Method chaining complete!")
print(f"\nTest R²: {stats2[(stats2['metric'] == 'r_squared') & (stats2['split'] == 'test')]['value'].values[0]:.4f}")

Method chaining complete!

Test R²: -1.7005


In [26]:
# Extract components
model_fit = wf_ols_sk_fit.extract_fit_parsnip()
preprocessor = wf_ols_sk_fit.extract_preprocessor()
model_spec = wf_ols_sk_fit.extract_spec_parsnip()

print("Extracted components:")
print(f"  Model fit type: {type(model_fit)}")
print(f"  Preprocessor: {preprocessor}")
print(f"  Model spec: {model_spec.mode} with {model_spec.engine} engine")

# Access underlying sklearn model
sklearn_model = model_fit.fit_data["model"]
print(f"\n  Underlying model type: {type(sklearn_model)}")
print(f"  Number of coefficients: {len(sklearn_model.coef_)}")

Extracted components:
  Model fit type: <class 'py_parsnip.model_spec.ModelFit'>
  Preprocessor: target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1
  Model spec: regression with sklearn engine

  Underlying model type: <class 'sklearn.linear_model._base.LinearRegression'>
  Number of coefficients: 3


In [27]:
# Define different formulas (preprocessing strategies)
formulas = [
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1",                                                        # Minimal model
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month",                      # Add more features
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + totaltar",           # All features
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + I(mean_med_diesel_crack_input1_trade_month_lag2*mean_nwe_hsfo_crack_trade_month_lag1)",           # With interaction
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + I(mean_med_diesel_crack_input1_trade_month_lag2**2) + I(mean_nwe_hsfo_crack_trade_month_lag1**2)" # With polynomials
]

# Descriptive IDs for each formula
formula_ids = [
    "minimal",
    "medium",
    "full",
    "interaction",
    "polynomial"
]

print(f"Number of preprocessing strategies: {len(formulas)}")

Number of preprocessing strategies: 5
